# import

In [ ]:
import os
import sys
import math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


sys.path.append('../..')
import model.Baseline as baseline
import model.my_model as mymodel
import model.util_loss as util_ls
import model.util_dataloader as util_dl
import model.util_model as util_md
import data_process.util_data as util_dt


device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")

# local
# file_save_dir = r'C:\Users\29492\Desktop\server_data\main_REDD'
# file_load_dir = r'D:\dataset\DataFrames\REDD'
# server
file_save_dir = r'../../results'
file_load_dir = r'../../data'

dm = util_md.DictManager(file_save_dir)
mwm = util_md.ModelWeightManager(file_save_dir)

# prepare

In [ ]:
dataframe_path_origin = r'D:\dataset\DataFrames\REDD\house_1.csv'
house_origin_cols = ['oven-3', 'oven-4', 'refrigerator-5', 'dishwaser-6', 'lighting-9', 'washer_dryer-10', 'microwave-11',
                'bathroom_gfi-12', 'electric_heat-13', 'stove-14', 'lighting-17', 'lighting-18', 'washer_dryer-19', 'washer_dryer-20']


house_origin_cols_transfer = ['refrigerator-5', 'dishwaser-6', 'washer_dryer-10', 'microwave-11', 'bathroom_gfi-12']
on_threshold = [50, 50, 50, 50, 50]
dataframe_path_transfer = {r'house_2.csv':['refrigerator-9', 'dishwaser-10', 'washer_dryer-7', 'microwave-6','none'],
                           r'house_3.csv':['refrigerator-7','dishwaser-9','washer_dryer-14','microwave-16','bathroom_gfi-20'],
                           r'house_4.csv':['none','dishwaser-15','washer_dryer-7','none','bathroom_gfi-17'],
                           r'house_5.csv':['refrigerator-18','dishwaser-20','washer_dryer-8','microwave-3','none'],
                           r'house_6.csv':['refrigerator-8','dishwaser-9','washer_dryer-4','none','bathroom_gfi-7']}

'''

'''


min_val = 0
max_val = 8781
loss_factor=max_val-min_val

metric_dict_one2one = {'MAE(apps)':util_ls.MAE(single_class=True),
                'SAE':util_ls.SignalAggregateError(single_class=True, period_len=450),
                'MAE(offon)':util_ls.MAE_off_on(single_class=True)}

metric_dict_one2all = {'MAE(apps)':util_ls.MAE(single_class=False),
                       'MAE(mean)':util_ls.MAE(single_class=True),
                'SAE(apps)':util_ls.SignalAggregateError(single_class=False, period_len=450),
                'SAE(mean)':util_ls.SignalAggregateError(single_class=True, period_len=450),
                'MAE(offon)':util_ls.MAE_off_on(single_class=False)}

# Main

## DAE

In [ ]:
eval_b_s = 450*10
sliding_window_len = 599
in_seq_l = sliding_window_len
out_dim = 599


for (filename, cols) in dataframe_path_transfer.items():
    path = os.path.join(file_load_dir, filename)
    print('-'*40, f'file path:{path}', sep='\n')
    tmp_df = pd.read_csv(path)
    main = tmp_df.filter(regex='^main').sum(axis=1).to_numpy()
    main = (main - min_val)/(max_val - min_val)
    for origin_app, transfer_app, thre in zip(house_origin_cols_transfer, cols, on_threshold):
        print('-'*20)
        print(f'Matching A[{origin_app}] to B[{transfer_app}]')
        try:
            app = tmp_df[transfer_app]
            app = (app - min_val)/(max_val - min_val)
            print(f'--- {transfer_app} found')
        except KeyError:
            print(f'!!! {transfer_app} no found')
            continue

        if not origin_app == 'refrigerator-5':
            continue

        w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=app, window_size_1=sliding_window_len,
                                     window_size_2=sliding_window_len, offset=0)
        w_main = np.expand_dims(w_main, axis=1)
        w_apps = np.expand_dims(w_apps, axis=1)
        w_apps_on = (w_apps>(thre-min_val)/(max_val-min_val)).astype('int')
        dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                      torch.from_numpy(w_apps).to(torch.float32),
                                      torch.from_numpy(w_apps_on).to(torch.float32))
        dataset.describe()
        testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=eval_b_s, shuffle_flag=False)

        model = baseline.DAE(in_seq_l)
        mwm.load_model_weight(model, '{dataset}_{model}_{appname}_{date}'
                              .format(dataset='REDD',model='DAE',appname=origin_app,date='1229'))

        score_test = util_md.train_val(device, model, valset=testset, loss_factor=loss_factor,
                        return_metric_dict=metric_dict_one2one)


## s2p

In [ ]:
eval_b_s = 450*10
sliding_window_len = 599
in_seq_l = sliding_window_len
out_dim = 1


for (filename, cols) in dataframe_path_transfer.items():
    path = os.path.join(file_load_dir, filename)
    print('-'*40, f'file path:{path}', sep='\n')
    tmp_df = pd.read_csv(path)
    tmp_df = tmp_df.dropna()
    main = tmp_df.filter(regex='^main').sum(axis=1).to_numpy()
    main = (main - min_val)/(max_val - min_val)
    for origin_app, transfer_app, thre in zip(house_origin_cols_transfer, cols, on_threshold):
        print('-'*20)
        print(f'Matching A[{origin_app}] to B[{transfer_app}]')
        try:
            app = tmp_df[transfer_app]
            app = (app - min_val)/(max_val - min_val)
            print(f'--- {transfer_app} found')
        except KeyError:
            print(f'!!! {transfer_app} no found')
            continue

        w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=app, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
        w_main = np.expand_dims(w_main, axis=1)
        w_apps = np.expand_dims(w_apps, axis=1)
        w_apps_on = (w_apps>(thre-min_val)/(max_val-min_val)).astype('int')
        dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                      torch.from_numpy(w_apps).to(torch.float32),
                                      torch.from_numpy(w_apps_on).to(torch.float32))
        dataset.describe()
        testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=eval_b_s, shuffle_flag=False)

        model = baseline.Seq2Point(in_seq_l, 1)
        mwm.load_model_weight(model, '{dataset}_{model}_{appname}_{date}'
                              .format(dataset='REDD',model='S2P',appname=origin_app,date='1228'))

        score_test = util_md.train_val(device, model, valset=testset, loss_factor=loss_factor,
                        return_metric_dict=metric_dict_one2one)


## S2S

In [ ]:
eval_b_s = 450*10
sliding_window_len = 599
in_seq_l = sliding_window_len
out_dim = 599


for (filename, cols) in dataframe_path_transfer.items():
    path = os.path.join(file_load_dir, filename)
    print('-'*40, f'file path:{path}', sep='\n')
    tmp_df = pd.read_csv(path)
    main = tmp_df.filter(regex='^main').sum(axis=1).to_numpy()
    main = (main - min_val)/(max_val - min_val)
    for origin_app, transfer_app, thre in zip(house_origin_cols_transfer, cols, on_threshold):
        print('-'*20)
        print(f'Matching A[{origin_app}] to B[{transfer_app}]')
        try:
            app = tmp_df[transfer_app]
            app = (app - min_val)/(max_val - min_val)
            print(f'--- {transfer_app} found')
        except KeyError:
            print(f'!!! {transfer_app} no found')
            continue

        w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=app, window_size_1=sliding_window_len,
                                     window_size_2=sliding_window_len, offset=0)
        w_main = np.expand_dims(w_main, axis=1)
        # w_apps = np.expand_dims(w_apps, axis=1)
        w_apps_on = (w_apps>(thre-min_val)/(max_val-min_val)).astype('int')
        dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                      torch.from_numpy(w_apps).to(torch.float32),
                                      torch.from_numpy(w_apps_on).to(torch.float32))
        dataset.describe()
        testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=eval_b_s, shuffle_flag=False)

        model = baseline.Seq2Seq(in_seq_l, in_seq_l)
        mwm.load_model_weight(model, '{dataset}_{model}_{appname}_{date}'
                              .format(dataset='REDD',model='S2S',appname=origin_app,date='1228'))

        score_test = util_md.train_val(device, model, valset=testset, loss_factor=loss_factor,
                        return_metric_dict=metric_dict_one2one)


## mark

In [ ]:
eval_b_s = 450*10
sliding_window_len = 599
in_seq_l = sliding_window_len
out_dim = len(house_origin_cols)
on_threshold = np.array([50, 50, 50, 50, 20, 50, 50, 50, 50, 50, 20, 20, 50, 50])


for (filename, cols) in dataframe_path_transfer.items():
    path = os.path.join(file_load_dir, filename)
    print('-'*40, f'file path:{path}', sep='\n')
    tmp_df = pd.read_csv(path)
    main = tmp_df.filter(regex='^main').sum(axis=1).to_numpy()
    main = (main - min_val)/(max_val - min_val)
    apps = np.zeros((len(main), len(house_origin_cols)))

    for origin_app, transfer_app in zip(house_origin_cols_transfer, cols):
        print('-'*20)
        print(f'Matching A[{origin_app}] to B[{transfer_app}]')
        try:
            app = tmp_df[transfer_app]
            app = (app - min_val)/(max_val - min_val)
            print(f'--- {transfer_app} found')
            col_index = house_origin_cols.index(origin_app)
            apps[:,col_index] = app
        except KeyError:
            print(f'!!! {transfer_app} no found')

    w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=apps, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
    w_main = np.expand_dims(w_main, axis=1)
    w_apps_sum = np.zeros((len(w_apps),1))
    w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
    dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                            torch.from_numpy(w_apps_on).to(torch.float32),
                            torch.from_numpy(w_apps_sum).to(torch.float32),
                            torch.from_numpy(w_apps).to(torch.float32))
    dataset.describe()
    testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=eval_b_s, shuffle_flag=False)

    model = mymodel.Model_final(in_seq_l, out_dim)
    mwm.load_model_weight(model,'{dataset}_{model}_{appname}_{date}'
                          .format(dataset='REDD',model='Final',appname='all',date=f'1228-{2}'))
    score_test, _ = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, valset=testset, loss_reg_factor=loss_factor,
                    return_metric_reg_dict=metric_dict_one2all)

    for origin_app, transfer_app in zip(house_origin_cols_transfer, cols):
        try:
            print(f'--- {transfer_app} found')
            col_index = house_origin_cols.index(origin_app)
            print(score_test['val']['MAE(apps)'][0, col_index])
            print(score_test['val']['SAE(apps)'][0, col_index])
            print(score_test['val']['MAE(offon)'][0][:, col_index])
        except KeyError:
            print(f'!!! {transfer_app} no found')


In [ ]:
score_test['val']

## SGN

In [ ]:
eval_b_s = 450*20
sliding_window_len = 599
in_seq_l = sliding_window_len
out_dim = 1


result_df_mae = pd.DataFrame(index=house_origin_cols_transfer, columns=dataframe_path_transfer.keys())
result_df_sae = pd.DataFrame(index=house_origin_cols_transfer, columns=dataframe_path_transfer.keys())

for (filename, cols) in dataframe_path_transfer.items():
    path = os.path.join(file_load_dir, filename)
    print('-'*40, f'file path:{path}', sep='\n')
    tmp_df = pd.read_csv(path)
    tmp_df = tmp_df.dropna()
    main = tmp_df.filter(regex='^main').sum(axis=1).to_numpy()
    main = (main - min_val)/(max_val - min_val)
    for origin_app, transfer_app, thre in zip(house_origin_cols_transfer, cols, on_threshold):
        print('-'*20)
        print(f'Matching A[{origin_app}] to B[{transfer_app}]')
        try:
            app = tmp_df[transfer_app]
            app = (app - min_val)/(max_val - min_val)
            print(f'--- {transfer_app} found')
        except KeyError:
            print(f'!!! {transfer_app} no found')
            continue

        w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=app, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
        w_main = np.expand_dims(w_main, axis=1)
        w_apps = np.expand_dims(w_apps, axis=1)
        w_apps_on = (w_apps>(thre-min_val)/(max_val-min_val)).astype('int')
        dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                      torch.from_numpy(w_apps).to(torch.float32),
                                      torch.from_numpy(w_apps_on).to(torch.float32))
        dataset.describe()
        testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=eval_b_s, shuffle_flag=False)

        model = baseline.SGN(in_seq_l)
        mwm.load_model_weight(model, '{dataset}_{model}_{appname}_{date}'
                              .format(dataset='REDD',model='SGN',appname=origin_app,date='0423'))

        score_test, _ = util_md.train_val_DoubleTask(device, model, valset=testset, loss_reg_factor=loss_factor,
                        return_metric_reg_dict=metric_dict_one2one)
        
        result_df_mae.loc[origin_app,filename] = score_test['val']['MAE(apps)'][0]
        result_df_sae.loc[origin_app,filename] = score_test['val']['SAE'][0][0]

result_df_mae['Ave'] = result_df_mae.mean(axis=1)
result_df_sae['Ave'] = result_df_sae.mean(axis=1)



print(result_df_mae)
print(result_df_sae)
print(result_df_mae['Ave'].mean())
print(result_df_sae['Ave'].mean())

# sub